In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Paraná - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_PR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.284196,0.300568,332.705560,2.027147e+08,1.060612e+07,17.589536,1.851788e+08,0.776871,189.804
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.277973,0.301710,332.945945,2.029477e+08,1.061597e+07,17.592548,1.852326e+08,0.776975,174.713
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.271750,0.302645,335.222130,2.031806e+08,1.062582e+07,17.595559,1.852864e+08,0.777079,180.801
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.265527,0.303413,334.785737,2.034136e+08,1.063567e+07,17.598570,1.853401e+08,0.777183,180.172
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.259304,0.304034,335.050185,2.036466e+08,1.064551e+07,17.601581,1.853939e+08,0.777287,185.597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.549782,616.073841,NaN,NaN,NaN,NaN,NaN,412.177
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.546864,614.423079,NaN,NaN,NaN,NaN,NaN,337.056
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544541,613.772150,NaN,NaN,NaN,NaN,NaN,340.018
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.540629,614.761140,NaN,NaN,NaN,NaN,NaN,383.477


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.200588,-0.990872,-1.688922,-1.661059,-2.360392,-1.926984,-1.800632,-2.165316
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.186349,-0.972615,-1.684978,-1.644277,-2.312924,-1.907242,-1.780025,-2.131422
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.172110,-0.957666,-1.647627,-1.627495,-2.265457,-1.887500,-1.759417,-2.097529
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.157871,-0.945388,-1.654788,-1.610713,-2.217990,-1.867758,-1.738809,-2.063636
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.143632,-0.935451,-1.650449,-1.593931,-2.170523,-1.848016,-1.718202,-2.029743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.826445,0.770388,1.208142,1.129314,-0.514291,0.743016,0.899324,0.733248
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.840099,0.821622,1.241242,1.120207,-0.507255,0.729140,0.888262,0.688083
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.853753,0.859766,1.313851,1.111101,-0.500218,0.715264,0.877199,0.642919
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.867406,0.897253,1.363920,1.101994,-0.493182,0.701389,0.866137,0.597754


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      185.712
1      170.875
2      204.477
3      173.662
4      162.633
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Paraná - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.200588,-0.990872,-1.688922,-1.661059,-2.360392,-1.926984,-1.800632,-2.165316
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.186349,-0.972615,-1.684978,-1.644277,-2.312924,-1.907242,-1.780025,-2.131422
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.172110,-0.957666,-1.647627,-1.627495,-2.265457,-1.887500,-1.759417,-2.097529
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.157871,-0.945388,-1.654788,-1.610713,-2.217990,-1.867758,-1.738809,-2.063636
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.143632,-0.935451,-1.650449,-1.593931,-2.170523,-1.848016,-1.718202,-2.029743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.130907,0.732054,0.692024,1.202413,-0.413954,0.937022,1.076355,1.563435
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.076074,0.695547,0.722303,1.204670,-0.429516,0.934746,1.074731,1.542097
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.021241,0.667742,0.729289,1.206927,-0.445078,0.932469,1.073107,1.520760
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.033592,0.634897,0.709776,1.209183,-0.460640,0.930193,1.071484,1.499423


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      185.712
1      170.875
2      204.477
3      173.662
4      162.633
        ...   
157    280.736
158    352.332
159    280.856
160    319.397
161    295.932
Name: Paraná - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,-1.195259,1.373912,0.887005,0.826002,0.454561,0.988596,0.979507,0.670392
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,-1.109331,1.418215,0.875394,0.842968,0.424602,0.981838,0.982236,0.678142
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,-1.023403,1.454067,0.855534,0.859935,0.394643,0.975081,0.984964,0.685892
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,-0.937476,1.490026,0.842489,0.876901,0.364683,0.968323,0.987693,0.693642
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,-0.851548,1.513267,0.820774,0.893867,0.334724,0.961566,0.990422,0.701392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.826445,0.770388,1.208142,1.129314,-0.514291,0.743016,0.899324,0.733248
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.840099,0.821622,1.241242,1.120207,-0.507255,0.729140,0.888262,0.688083
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.853753,0.859766,1.313851,1.111101,-0.500218,0.715264,0.877199,0.642919
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.867406,0.897253,1.363920,1.101994,-0.493182,0.701389,0.866137,0.597754


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=True), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1840839841, 2179890005, 3536929403, 1885310629, 2556953278, 2871941356, 3240275501, 295572191, 1453397224, 2566963376, 2998105769, 3431109688, 492693902, 2380301410, 1217105149, 244990869, 1614766523, 2393467168, 2049014281, 3116227224, 3520946411, 730390096, 2794363846, 3280042938, 2004110875, 4145939247, 3759264120, 2948732100, 4148739195, 4091489636, 1385970621, 2217398589, 897405071, 2271919277, 2109639493, 3090946494, 3549268090, 171836922, 4135542180, 3458378978, 1582585574, 454238531, 3664194983, 4206016640, 1786199901, 585212235, 3052011793, 169934289, 2069161722, 421421402]


Step: 0 ___________________________________________
val_loss: 243.70404052734375
winner_seed: 1840839841


Step: 1 ___________________________________________
val_loss: 256.6778259277344


Step: 2 ___________________________________________
val_loss: 357.9258117675781


Step: 3 ___________________________________________
val_loss: 390.7344055175781


Step: 4 ___________________________________________
va

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 200ms/step - loss: 75218.1406 - val_loss: 150234.2500
Epoch 2/10000
4/4 [==============================] - 0s 12ms/step - loss: 406251.9375 - val_loss: 1674918.2500
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 1057711.5000 - val_loss: 15778.9824
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 14723.9209 - val_loss: 2671.2388
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 2264.4094 - val_loss: 1750.2771
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 1473.8101 - val_loss: 1017.1024
Epoch 7/10000
4/4 [==============================] - 0s 11ms/step - loss: 1809.6287 - val_loss: 1407.3918
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 2415.8967 - val_loss: 1932.2902
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 1409.2272 - val_loss: 1773.4476
Epoch 10/10000
4/4 [============

4/4 [==============================] - 0s 10ms/step - loss: 744.8641 - val_loss: 619.1678
Epoch 156/10000
4/4 [==============================] - 0s 10ms/step - loss: 583.2712 - val_loss: 546.2508
Epoch 157/10000
4/4 [==============================] - 0s 10ms/step - loss: 517.1166 - val_loss: 476.8724
Epoch 158/10000
4/4 [==============================] - 0s 10ms/step - loss: 448.1550 - val_loss: 517.1181
Epoch 159/10000
4/4 [==============================] - 0s 10ms/step - loss: 530.9332 - val_loss: 534.7252
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 590.1633 - val_loss: 512.4380
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 439.8112 - val_loss: 491.0622
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 518.2333 - val_loss: 969.4911
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 613.4087 - val_loss: 516.4610
Epoch 164/10000
4/4 [==============================] - 0s 10ms

4/4 [==============================] - 0s 10ms/step - loss: 413.1885 - val_loss: 452.8033
Epoch 310/10000
4/4 [==============================] - 0s 11ms/step - loss: 442.4056 - val_loss: 473.1278
Epoch 311/10000
4/4 [==============================] - 0s 10ms/step - loss: 391.9693 - val_loss: 460.9496
Epoch 312/10000
4/4 [==============================] - 0s 10ms/step - loss: 435.4078 - val_loss: 472.0737
Epoch 313/10000
4/4 [==============================] - 0s 10ms/step - loss: 521.3391 - val_loss: 401.4609
Epoch 314/10000
4/4 [==============================] - 0s 10ms/step - loss: 404.0579 - val_loss: 468.9235
Epoch 315/10000
4/4 [==============================] - 0s 10ms/step - loss: 366.4543 - val_loss: 459.3821
Epoch 316/10000
4/4 [==============================] - 0s 10ms/step - loss: 411.5718 - val_loss: 432.0930
Epoch 317/10000
4/4 [==============================] - 0s 10ms/step - loss: 412.3087 - val_loss: 550.1894
Epoch 318/10000
4/4 [==============================] - 0s 10ms

4/4 [==============================] - 0s 10ms/step - loss: 363.6568 - val_loss: 475.5524
Epoch 464/10000
4/4 [==============================] - 0s 10ms/step - loss: 339.7697 - val_loss: 513.8130
Epoch 465/10000
4/4 [==============================] - 0s 10ms/step - loss: 394.0946 - val_loss: 489.0429
Epoch 466/10000
4/4 [==============================] - 0s 10ms/step - loss: 417.2346 - val_loss: 448.9554
Epoch 467/10000
4/4 [==============================] - 0s 10ms/step - loss: 378.8669 - val_loss: 450.7259
Epoch 468/10000
4/4 [==============================] - 0s 10ms/step - loss: 333.6973 - val_loss: 424.7209
Epoch 469/10000
4/4 [==============================] - 0s 10ms/step - loss: 346.6341 - val_loss: 424.8231
Epoch 470/10000
4/4 [==============================] - 0s 10ms/step - loss: 396.4686 - val_loss: 438.7632
Epoch 471/10000
4/4 [==============================] - 0s 10ms/step - loss: 359.1337 - val_loss: 468.7559
Epoch 472/10000
4/4 [==============================] - 0s 10ms

4/4 [==============================] - 0s 10ms/step - loss: 351.3396 - val_loss: 471.4928
Epoch 618/10000
4/4 [==============================] - 0s 10ms/step - loss: 321.9105 - val_loss: 579.6962
Epoch 619/10000
4/4 [==============================] - 0s 10ms/step - loss: 337.8352 - val_loss: 686.5865
Epoch 620/10000
4/4 [==============================] - 0s 10ms/step - loss: 367.6457 - val_loss: 483.5350
Epoch 621/10000
4/4 [==============================] - 0s 10ms/step - loss: 391.4114 - val_loss: 474.8911
Epoch 622/10000
4/4 [==============================] - 0s 10ms/step - loss: 334.5118 - val_loss: 452.4279
Epoch 623/10000
4/4 [==============================] - 0s 10ms/step - loss: 303.1934 - val_loss: 487.4861
Epoch 624/10000
4/4 [==============================] - 0s 10ms/step - loss: 329.6306 - val_loss: 579.7037
Epoch 625/10000
4/4 [==============================] - 0s 10ms/step - loss: 319.0348 - val_loss: 608.6375
Epoch 626/10000
4/4 [==============================] - 0s 10ms

...

4/4 [==============================] - 0s 10ms/step - loss: 115.7637 - val_loss: 303.9978
Epoch 2295/10000
4/4 [==============================] - 0s 10ms/step - loss: 191.2903 - val_loss: 278.1478
Epoch 2296/10000
4/4 [==============================] - 0s 10ms/step - loss: 177.5606 - val_loss: 288.2604
Epoch 2297/10000
4/4 [==============================] - 0s 10ms/step - loss: 214.3081 - val_loss: 375.0159
Epoch 2298/10000
4/4 [==============================] - 0s 10ms/step - loss: 186.2181 - val_loss: 324.6641
Epoch 2299/10000
4/4 [==============================] - 0s 10ms/step - loss: 169.7855 - val_loss: 289.5028
Epoch 2300/10000
4/4 [==============================] - 0s 10ms/step - loss: 202.5020 - val_loss: 459.5045
Epoch 2301/10000
4/4 [==============================] - 0s 10ms/step - loss: 192.9238 - val_loss: 475.0379
Epoch 2302/10000
4/4 [==============================] - 0s 10ms/step - loss: 209.0911 - val_loss: 472.8983
Epoch 2303/10000
4/4 [==============================] 

4/4 [==============================] - 0s 10ms/step - loss: 152.2915 - val_loss: 395.6039
Epoch 2447/10000
4/4 [==============================] - 0s 10ms/step - loss: 196.6545 - val_loss: 412.2733
Epoch 2448/10000
4/4 [==============================] - 0s 10ms/step - loss: 167.1826 - val_loss: 501.6718
Epoch 2449/10000
4/4 [==============================] - 0s 10ms/step - loss: 183.1280 - val_loss: 437.6813
Epoch 2450/10000
4/4 [==============================] - 0s 10ms/step - loss: 275.7953 - val_loss: 402.0460
Epoch 2451/10000
4/4 [==============================] - 0s 10ms/step - loss: 182.7727 - val_loss: 343.4623
Epoch 2452/10000
4/4 [==============================] - 0s 10ms/step - loss: 137.4825 - val_loss: 319.7145
Epoch 2453/10000
4/4 [==============================] - 0s 10ms/step - loss: 174.7219 - val_loss: 407.6690
Epoch 2454/10000
4/4 [==============================] - 0s 10ms/step - loss: 159.8655 - val_loss: 421.9185
Epoch 2455/10000
4/4 [==============================] 

4/4 [==============================] - 0s 10ms/step - loss: 173.3702 - val_loss: 397.9781
Epoch 2599/10000
4/4 [==============================] - 0s 10ms/step - loss: 150.0957 - val_loss: 358.0107
Epoch 2600/10000
4/4 [==============================] - 0s 10ms/step - loss: 110.9379 - val_loss: 333.3766
Epoch 2601/10000
4/4 [==============================] - 0s 10ms/step - loss: 181.9315 - val_loss: 384.3506
Epoch 2602/10000
4/4 [==============================] - 0s 10ms/step - loss: 164.7368 - val_loss: 316.9928
Epoch 2603/10000
4/4 [==============================] - 0s 10ms/step - loss: 188.1884 - val_loss: 349.0974
Epoch 2604/10000
4/4 [==============================] - 0s 10ms/step - loss: 179.1726 - val_loss: 346.6691
Epoch 2605/10000
4/4 [==============================] - 0s 10ms/step - loss: 213.8819 - val_loss: 334.5007
Epoch 2606/10000
4/4 [==============================] - 0s 10ms/step - loss: 114.3824 - val_loss: 407.9640
Epoch 2607/10000
4/4 [==============================] 

4/4 [==============================] - 0s 10ms/step - loss: 609.7319 - val_loss: 497.0120
Epoch 2751/10000
4/4 [==============================] - 0s 10ms/step - loss: 256.0423 - val_loss: 517.7812
Epoch 2752/10000
4/4 [==============================] - 0s 10ms/step - loss: 287.0658 - val_loss: 541.6257
Epoch 2753/10000
4/4 [==============================] - 0s 10ms/step - loss: 279.1165 - val_loss: 553.1062
Epoch 2754/10000
4/4 [==============================] - 0s 10ms/step - loss: 275.8312 - val_loss: 474.2137
Epoch 2755/10000
4/4 [==============================] - 0s 10ms/step - loss: 253.0006 - val_loss: 470.8405
Epoch 2756/10000
4/4 [==============================] - 0s 10ms/step - loss: 243.4060 - val_loss: 442.7365
Epoch 2757/10000
4/4 [==============================] - 0s 10ms/step - loss: 239.4015 - val_loss: 424.2033
Epoch 2758/10000
4/4 [==============================] - 0s 10ms/step - loss: 231.9952 - val_loss: 430.9724
Epoch 2759/10000
4/4 [==============================] 

4/4 [==============================] - 0s 10ms/step - loss: 238.4678 - val_loss: 355.5716
Epoch 2903/10000
4/4 [==============================] - 0s 10ms/step - loss: 191.8863 - val_loss: 469.7350
Epoch 2904/10000
4/4 [==============================] - 0s 10ms/step - loss: 261.9969 - val_loss: 345.0142
Epoch 2905/10000
4/4 [==============================] - 0s 10ms/step - loss: 220.5598 - val_loss: 583.5984
Epoch 2906/10000
4/4 [==============================] - 0s 11ms/step - loss: 246.6601 - val_loss: 317.9729
Epoch 2907/10000
4/4 [==============================] - 0s 10ms/step - loss: 237.5587 - val_loss: 403.9417
Epoch 2908/10000
4/4 [==============================] - 0s 10ms/step - loss: 225.7116 - val_loss: 307.4076
Epoch 2909/10000
4/4 [==============================] - 0s 10ms/step - loss: 176.6213 - val_loss: 297.3999
Epoch 2910/10000
4/4 [==============================] - 0s 10ms/step - loss: 211.8385 - val_loss: 340.2118
Epoch 2911/10000
4/4 [==============================] 

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,299.027771,303.692841,299.515411,300.589172,300.287109,299.588959,302.68634,322.053558,322.716827,322.510284,323.522003,324.102081,306.731995,323.607086,324.11264,323.395752,322.300903,311.243835,323.50238,324.180511,323.734985,323.257538,323.705963,323.04068,323.746185,322.744415,322.983948,323.284882,324.061218,325.138885
Target,354.558,360.106,350.123,288.254,301.687,268.901,274.431,298.219,328.955,335.039,260.723,366.929,353.426,344.937,302.082,294.467,336.029,278.117,323.005,291.453,300.203,317.058,314.142,301.878,358.797,356.169,316.188,346.214,323.401,238.466
Error,55.530243,56.413147,50.607574,12.335175,1.399902,30.687958,28.255341,23.834564,6.238159,12.528717,62.799011,42.826904,46.694,21.329926,22.03064,28.928741,13.728088,33.126831,0.497375,32.727509,23.531982,6.199524,9.563965,21.162689,35.050812,33.424591,6.795959,22.929108,0.660217,86.672882


In [38]:
display(mae)
display(mape)

27.617054

0.08994549

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[3720.2922]] - Target[3787.925]| =  Error: [[67.63281]]; MAPE:[[0.01785485]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[3852.8145]] - Target[3756.797]| =  Error: [[96.017334]]; MAPE:[[0.0255583]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[1941.9595]] - Target[1939.2350000000001]| =  Error: [[2.7244873]]; MAPE:[[0.00140493]]


[array([[67.63281]], dtype=float32),
 array([[96.017334]], dtype=float32),
 array([[2.7244873]], dtype=float32)]

55.45821

0.014939357